## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-03-02-01-18-00 +0000,48hills,Noise Pop Diary: Shannon Shaw came sans Clams;...,https://48hills.org/2026/03/noise-pop-diary-sh...
1,2026-03-02-01-17-13 +0000,nypost,Trapped hot air balloon passengers saved in he...,https://nypost.com/2026/03/01/us-news/trapped-...
2,2026-03-02-01-15-43 +0000,nypost,Family speaks out after FBI launched terror pr...,https://nypost.com/2026/03/01/us-news/oc-famil...
3,2026-03-02-01-12-37 +0000,wapo,"Live updates: U.S. military says 1,000 targets...",https://www.washingtonpost.com/world/2026/03/0...
4,2026-03-02-01-12-37 +0000,wapo,Video shows explosion near Erbil International...,https://www.washingtonpost.com


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2587/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
47,iran,87
73,strikes,24
76,trump,24
447,khamenei,22
70,israel,21


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
187,2026-03-01-10-59-34 +0000,wapo,Iran’s supreme leader killed in U.S.-Israeli a...,https://www.washingtonpost.com/world/2026/02/2...,205
153,2026-03-01-15-31-00 +0000,nyt,Israel launches new military strikes as Iran r...,https://www.nytimes.com/live/2026/03/01/world/...,180
215,2026-03-01-05-44-12 +0000,nypost,Trump warns US will strike Iran with ‘force th...,https://nypost.com/2026/03/01/world-news/trump...,180
221,2026-03-01-04-56-41 +0000,nypost,Trump says diplomatic agreement with Iran will...,https://nypost.com/2026/02/28/world-news/trump...,175
57,2026-03-01-22-24-27 +0000,bbc,Nine dead in missile attack on Israel as Iran ...,https://www.bbc.com/news/articles/c363zkp1pgxo...,160


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
187,205,2026-03-01-10-59-34 +0000,wapo,Iran’s supreme leader killed in U.S.-Israeli a...,https://www.washingtonpost.com/world/2026/02/2...
47,87,2026-03-01-22-57-46 +0000,nypost,Trump warns Iranian regime to surrender or fac...,https://nypost.com/2026/03/01/world-news/trump...
132,54,2026-03-01-17-32-51 +0000,nyt,"Ayatollah Ali Khamenei, Hard-Line Cleric Who M...",https://www.nytimes.com/2026/02/28/world/middl...
80,51,2026-03-01-21-09-38 +0000,cbc,Hundreds of thousands stranded as war shuts do...,https://www.cbc.ca/news/world/middle-east-iran...
2,41,2026-03-02-01-15-43 +0000,nypost,Family speaks out after FBI launched terror pr...,https://nypost.com/2026/03/01/us-news/oc-famil...
48,38,2026-03-01-22-54-12 +0000,nypost,Mom reveals horrific new details after daughte...,https://nypost.com/2026/03/01/us-news/reseda-g...
229,33,2026-03-01-02-53-00 +0000,wsj,Democrats in Congress led the charge to releas...,https://www.wsj.com/politics/policy/epstein-fi...
215,26,2026-03-01-05-44-12 +0000,nypost,Trump warns US will strike Iran with ‘force th...,https://nypost.com/2026/03/01/world-news/trump...
41,25,2026-03-01-23-18-27 +0000,bbc,Watch: Retaliations continue on day two of US-...,https://www.bbc.com/news/videos/cvgk8vj56v1o?a...
77,25,2026-03-01-21-38-23 +0000,nypost,GOP civil war in Texas likely to lead to US Se...,https://nypost.com/2026/03/01/us-news/us-senat...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
